In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# downlaod deepmind's pretrained language model
# !wget -O deepmind_assets/language_perceiver_io_bytes.pickle https://storage.googleapis.com/perceiver_io/language_perceiver_io_bytes.pickle

In [3]:
from perceiver_io.perceiver_lm import PerceiverLM

import os, sys
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import transformers

from deepmind_assets import bytes_tokenizer
import numpy as np
import scipy.sparse as sp
from tqdm.notebook import tqdm
import scipy.sparse as sp
import xclib.evaluation.xc_metrics as xc_metrics
from utils import csr_to_pad_tensor, ToD, read_sparse_mat, XCMetrics, _c
from torch.nn.utils.rnn import pad_sequence

# The tokenizer is just UTF-8 encoding (with an offset)
tokenizer = bytes_tokenizer.BytesTokenizer()

In [4]:
command = "--dataset cifar10"

import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--project', default='PerceiverIO')
parser.add_argument('--dataset', default='EURLex-4K')
parser.add_argument('--device', type=str, default='cuda:0')

args = parser.parse_args(command.split())

In [6]:
args.expname = f'{args.project}-image'
args.maxlen = 2048
args.vocab_size = 262
args.embed_dim = 768
args.num_latents = 256

args.n_epochs = 10
args.lr = 1e-4
args.bsz = 16
args.dropout = 0.5
args.warmup = 0.1
args.loss_with_logits = True
args.amp = False
args.eval_interval = 2

args.per_label_task = False
args.per_token_decoder = False

OUT_DIR = f'Results/{args.expname}/{args.dataset}'
os.makedirs(OUT_DIR, exist_ok=True)

In [7]:
args.img_size = 32
if args.dataset == 'tiny-imagenet':
    args.img_size = 64
elif args.dataset == 'stl10':
    args.img_size = 96
elif args.dataset == 'cifar10':
    args.img_size = 32
    
transform_train = transforms.Compose([
    transforms.Resize(args.img_size),
    transforms.RandomCrop(args.img_size, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.Resize(args.img_size),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [8]:
if args.dataset == 'cifar10':
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=False, transform=transform_train)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=False, transform=transform_test)
    args.numy = 10
elif args.dataset == 'tiny-imagenet':
    fix_tin_val_folder('./data/tiny-imagenet-200/val')
    trainset = torchvision.datasets.ImageFolder(root='./data/tiny-imagenet-200/train', transform=transform_train)
    testset = torchvision.datasets.ImageFolder(root='./data/tiny-imagenet-200/val', transform=transform_test)
    args.numy = 200
elif args.dataset == 'stl10':
    trainset = torchvision.datasets.STL10(root='./data', split='train', download=False, transform=transform_train)
    testset = torchvision.datasets.STL10(root='./data', split='test', download=False, transform=transform_test)
    args.numy = 10

In [9]:
encoder = PerceiverLM(vocab_size=args.vocab_size, 
                      max_seq_len=args.maxlen, 
                      embedding_dim=args.embed_dim, 
                      num_latents=args.num_latents, 
                      latent_dim=1280, 
                      qk_out_dim=256, 
                      dropout=0,
                      num_self_attn_per_block=26, 
                      per_token_decoder=args.per_token_decoder, 
                      num_query_tasks=args.numy if args.per_label_task else 1)

encoder.load_pretrained("deepmind_assets/language_perceiver_io_bytes.pickle")

_IncompatibleKeys(missing_keys=['query_task_embedding.weight'], unexpected_keys=[])


In [10]:
from einops import rearrange, repeat
from einops.layers.torch import Rearrange

In [11]:
args.patch_height = 4
args.patch_width = 4
args.num_patches = (args.img_size // args.patch_height) * (args.img_size // args.patch_width)
args.patch_dim = 3 * args.patch_height * args.patch_width

In [12]:
## Sanity check for encoder with per_token_decoder=True

# input_str = "This is an incomplete sentence where some words are missing."
# input_tokens = tokenizer.to_int(input_str)

# # Mask " missing.". Note that the model performs much better if the masked chunk
# # starts with a space.
# input_tokens[51:60] = tokenizer.mask_token
# print("Tokenized string without masked bytes:")
# print(tokenizer.to_string(input_tokens))

# #@title Pad and reshape inputs
# inputs = input_tokens[None]
# input_mask = np.ones_like(inputs)

# def pad(max_sequence_length: int, inputs, input_mask):
#     input_len = inputs.shape[1]
#     assert input_len <= max_sequence_length
#     pad_len = max_sequence_length - input_len
#     padded_inputs = np.pad(
#       inputs,
#       pad_width=((0, 0), (0, pad_len)),
#       constant_values=tokenizer.pad_token)
#     padded_mask = np.pad(
#       input_mask,
#       pad_width=((0, 0), (0, pad_len)),
#       constant_values=0)
#     return padded_inputs, padded_mask

# inputs, input_mask = pad(args.maxlen, inputs, input_mask)

# encoder.eval()
# mask = torch.tensor(input_mask)
# input_ids = torch.tensor(inputs)
# out = encoder.forward(input_ids, mask)

# embs = out * mask.unsqueeze(-1) / mask.sum(dim=-1)

# logits = torch.matmul(out, encoder.token_embedding.weight.T) + encoder.decoder_token_bias
# masked_tokens_predictions = logits[0, 51:60].argmax(dim=-1)
# print("Greedy predictions:")
# print(masked_tokens_predictions)
# print()
# print("Predicted string:")
# print(tokenizer.to_string(masked_tokens_predictions.cpu().detach().numpy()))

In [13]:
class XMLDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels, tokenizer, maxlen):
        self.maxlen = maxlen
        self.input_ids = pad_sequence([torch.LongTensor(tokenizer.to_int(x)[:maxlen]) for x in inputs], batch_first=True, padding_value=0)
        self.input_mask = (self.input_ids != 0).long()
        self.labels = labels
            
    def __getitem__(self, index):
        return index
    
    def get_fts(self, indices, source='point'):
        input_mask = self.input_mask[indices]
        max_batch_seq_len = input_mask.sum(dim=-1).max()
        return {'input_ids': self.input_ids[indices, :max_batch_seq_len], 'input_mask': input_mask[:, :max_batch_seq_len]}
   
    def __len__(self):
        return self.labels.shape[0]
    
class XMLCollator():
    def __init__(self, dataset):
        self.numy = dataset.labels.shape[1]
        self.dataset = dataset
    
    def __call__(self, batch):
        ids = torch.LongTensor(batch)
        batch_data = {'batch_size': torch.LongTensor([len(batch)]),
                      'numy': torch.LongTensor([self.numy]),
                      'y': csr_to_pad_tensor(self.dataset.labels[ids], self.numy),
                      'ids': ids,
                      'xfts': self.dataset.get_fts(ids)
                     }
                
        return batch_data

In [14]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=args.bsz, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(testset, batch_size=args.bsz, shuffle=False, num_workers=4)

In [15]:
class Net(nn.Module):
    def __init__(self, encoder, args):
        super().__init__()
        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = args.patch_height, p2 = args.patch_width),
            nn.Linear(args.patch_dim, args.embed_dim),
        )
        self.encoder = encoder
        self.numy = args.numy
        self.dropout = nn.Dropout(args.dropout)
        if args.per_label_task:
            self.w = nn.Sequential(nn.Linear(args.embed_dim, 2*args.embed_dim), 
                                   nn.ReLU(), 
                                   nn.Linear(2*args.embed_dim, 1))
        else:
            self.w = nn.Linear(args.embed_dim, args.numy)
        
    def get_device(self):
        return list(self.parameters())[0].device
    
    def forward(self, b):
        patch_embs = self.to_patch_embedding(b)
        seq_len = patch_embs.size(1)
        batch_size = patch_embs.size(0)
        
        pos_ids = torch.arange(seq_len, device=patch_embs.device).view(1, -1)
        pos_embs = self.encoder.position_embedding(pos_ids)
        embs = patch_embs + pos_embs
        
        if args.per_token_decoder:
            query_embs = self.encoder.query_position_embedding(pos_ids).repeat(batch_size, 1, 1)
            query_mask = None
        else:
            query_embs = self.encoder.query_task_embedding.weight.repeat(batch_size, 1, 1)
            query_mask = None
            
        embs = self.encoder.perceiver(
            inputs=embs,
            query=query_embs,
            input_mask=None,
            query_mask=None
        )
        
        if self.encoder.per_token_decoder:
            embs = embs.mean(dim=1)
        else:
            embs = embs.squeeze()
    
        out = self.w(self.dropout(embs))
        return out.squeeze()
    
class OvABCELoss(nn.Module):
    def __init__(self, args, reduction='mean'):
        super(OvABCELoss, self).__init__()
        if args.loss_with_logits:
            self.criterion = torch.nn.BCEWithLogitsLoss(reduction=reduction)
        else:
            self.criterion = torch.nn.BCELoss(reduction=reduction)

    def forward(self, model, b):
        out = model(b)
        targets = torch.zeros((out.shape[0], out.shape[1]+1), device=out.device).scatter_(1, b['y']['inds'], 1)[:, :-1]
        loss = self.criterion(out, targets)
        return loss

In [16]:
net = Net(encoder, args)
criterion = nn.CrossEntropyLoss()

In [17]:
optims = [transformers.optimization.AdamW(net.parameters(), **{'lr': args.lr, 'eps': 1e-06, 'weight_decay': 0.01})]
total_steps = len(trainloader)*args.n_epochs
schedulers = [transformers.get_linear_schedule_with_warmup(optim, num_warmup_steps=int(args.warmup*total_steps), num_training_steps=total_steps) for optim in optims]

In [18]:
net.to(args.device);

In [28]:
def evaluate(net, testloader, epoch=-1):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    t = tqdm(testloader, desc='', leave=True)
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(t):
            inputs, targets = inputs.to(args.device), targets.to(args.device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            t.set_description(' '.join([str(batch_idx), str(len(testloader)), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (test_loss/(batch_idx+1), 100.*correct/total, correct, total)]))
    
    acc = 100.*correct/total
    loss = test_loss/(batch_idx+1)
    return loss, acc

In [19]:
best_acc = -100
for epoch in range(args.n_epochs):
    net.train()
    cum_loss = 0; ctr = 0
    t = tqdm(trainloader, desc='Epoch: 0, Loss: 0.0', leave=True)
          
    for b in t:        
        for optim in optims: optim.zero_grad()
        b = ToD({'input': b[0], 'label': b[1]}, args.device)
        with torch.cuda.amp.autocast(enabled=args.amp):
            out = net(b['input'])
        loss = criterion(out, b['label'])
        loss.backward()
        for optim in optims: optim.step()
        for sch in schedulers: sch.step()
        cum_loss += loss.item()
        ctr += 1
        t.set_description('Epoch: %d/%d, Loss: %.4E'%(epoch, args.n_epochs, (cum_loss/ctr)), refresh=True)
    
    print(f'mean loss after epoch {epoch}/{args.n_epochs}: {"%.4E"%(cum_loss/ctr)}', flush=True)
    if epoch%args.eval_interval == 0 or epoch == (args.n_epochs-1):
        test_loss, test_acc = evaluate(net, testloader)

        if test_acc > best_acc:
            best_acc = test_acc
            print(f'Found new best model with acc: {"%.2f"%best_acc}\n')
            with open(f'{OUT_DIR}/log.txt', 'a') as f:
                print(f'epoch: {epoch}, test acc: {test_acc}, train loss: {cum_loss/ctr}, test loss: {test_loss}', file=f)
            torch.save(net.state_dict(), f'{OUT_DIR}/model.pt')
    sys.stdout.flush()

Epoch: 0, Loss: 0.0:   0%|          | 0/3125 [00:00<?, ?it/s]

mean loss after epoch 0/10: 1.9841E+00


Epoch: 0, Loss: 0.0:   0%|          | 0/3125 [00:00<?, ?it/s]

mean loss after epoch 1/10: 1.6608E+00


Epoch: 0, Loss: 0.0:   0%|          | 0/3125 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [30]:
test_loss, test_acc = evaluate(net, trainloader)

  0%|          | 0/3125 [00:00<?, ?it/s]

In [35]:
num_params = 0
for p in model.parameters():
    num_params += np.prod(p.shape)

num_params

NameError: name 'model' is not defined